In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from utils import *

In [ ]:
expression_df, phenotype_df, gene_annot = get_geoparse()
phenotype_df["hours"] = (
    phenotype_df["title"]
    .str.extract(r"(\d+(?:\.\d+)?)\s*hours", expand=False)
    .astype(float)
)
phenotype_df["treated"] = ~phenotype_df["title"].str.contains(
    "negative control", case=False, na=False
)
phenotype_df = phenotype_df[['hours', 'treated']]

Precheck data for expression distribution

In [ ]:
plt.figure(figsize=(8, 7))
expression_df.hist(bins=100)
plt.title("Histogram of Samples")
plt.tight_layout()
plt.figure(figsize=(8, 7))
expression_df.boxplot()
plt.title("Boxplot of Samples")
plt.tight_layout()

PCA Analysis of Variance:
    Compare time and treatment to PC 1 & 2

In [ ]:
samples = phenotype_df.index
X = expression_df.T
assert all(X.index == phenotype_df.index)

pca = PCA(n_components=5)
X_pca = pca.fit_transform(X)
pca_df = pd.DataFrame(
    X_pca,
    index=X.index,
    columns=[f"PC{i+1}" for i in range(X_pca.shape[1])]
)

pca_df = pca_df.join(phenotype_df)
pca_df['hours_cat'] = pca_df['hours'].astype(str)

plt.figure(figsize=(6, 5))
sns.scatterplot(
    data=pca_df,
    x="PC1",
    y="PC2",
    hue="hours_cat",
    palette="viridis",
    s=80
)
plt.title("PCA of samples (colored by time)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 5))
sns.scatterplot(
    data=pca_df,
    x="PC1",
    y="PC2",
    hue="ctrl",
    style="ctrl",
    s=80
)
plt.title("PCA of samples (colored by treatment)")
plt.tight_layout()
plt.show()

Minimal Filtering:
    Only remove genes with pure background signal or low variance between samples

In [ ]:
MIN_EXPR = 4.5  # background cutoff
background_filter = (expression_df > MIN_EXPR).any(axis=1)
exp_filter = expression_df.loc[background_filter]
print(f"Genes after background filter: {exp_filter.shape[0]}")

MIN_VAR = 0.01
var_filter = exp_filter.var(axis=1) > MIN_VAR
exp_filter = exp_filter.loc[var_filter]
print(f"Genes after variance filter: {exp_filter.shape[0]}")
plt.hist(expression_df.values.flatten(), bins=100, alpha=0.5, label="All genes")
plt.hist(exp_filter.values.flatten(), bins=100, alpha=0.5, label="Filtered")
plt.legend()
plt.title("Expression distributions")
plt.show()

In [ ]:
results = fit_timecourse_ols(
     expr_gene_by_sample=exp_filter,
     pheno=phenotype_df,
     time_col="hours",
     type_col="treated",
     center_time=True,
 )

Volcano Plots

In [ ]:
res = results.copy()
res["abs_beta_treat"] = res["beta_treat"].abs()
res["abs_beta_interaction"] = res["beta_time_c:treat"].abs()
res["log10_fdr_treat"] = -np.log10(res["fdr_treat"] + 1e-300)
res["log10_fdr_interaction"] = -np.log10(res["fdr_interaction"] + 1e-300)
alpha = 0.05
alpha = 0.05
sig_treat = res['fdr_treat'] < alpha
sig_int = res['fdr_interaction'] < alpha
conditions = [
    (sig_treat & ~sig_int),   # only treatment
    (~sig_treat & sig_int),   # only interaction
    (sig_treat & sig_int),    # both
]
colors = ['red', 'blue', 'purple']
labels = ['Main effect only', 'Interaction only', 'Both']
res['category'] = np.select(conditions, labels, default='None')
color_map = {
    'Main effect only': 'red',
    'Interaction only': 'blue',
    'Both': 'purple',
    'None': 'gray'
}
plot_colors = res['category'].map(color_map)
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Treatment
axes[0].scatter(
    res["beta_treat"],
    res["log10_fdr_treat"],
    s=10,
    alpha=0.6,
    c=plot_colors,
)
axes[0].axvline(0, color="black", lw=1)
axes[0].axhline(-np.log10(0.05), color="red", ls="--", lw=1)
axes[0].set_xlabel("beta_treat")
axes[0].set_ylabel("-log10(FDR)")
axes[0].set_title("Treatment effect")
handles = [plt.Line2D([0], [0], marker='o', color='w', label=lab,
                      markerfacecolor=col, markersize=10) for lab, col in zip(labels, colors)]
handles.append(plt.Line2D([0], [0], marker='o', color='w', label='Not significant',
                          markerfacecolor='gray', markersize=10))
axes[0].legend(handles=handles, loc='upper left', frameon=True)

# Interaction
plt.scatter(
    res["beta_time_c:treat"],
    res["log10_fdr_interaction"],
    s=10,
    alpha=0.6,
    c=plot_colors,
)
axes[1].axvline(0, color="black", lw=1)
axes[1].axhline(-np.log10(0.05), color="red", ls="--", lw=1)
axes[1].set_xlabel("beta_time:treat")
axes[1].set_ylabel("-log10(FDR)")
axes[1].set_title("Time-dependent effect")
plt.tight_layout()
plt.show()

In [ ]:
hits = res[(res['fdr_treat'] < 0.05) | (res['fdr_interaction'] < 0.05)].copy()
main_only = res[(res['fdr_treat'] < 0.05) & (res['fdr_interaction'] >= 0.05)]
interaction_only = res[(res['fdr_interaction'] < 0.05) & (res['fdr_treat'] >= 0.05)]
both = res[(res['fdr_treat'] < 0.05) & (res['fdr_interaction'] < 0.05)]

print(f"Total responsive genes: {len(hits)}")
print(f"Main effect only: {len(main_only)}")
print(f"Interaction only: {len(interaction_only)}")
print(f"Both: {len(both)}")

Split Interaction into response type categories

In [ ]:
samples = phenotype_df[phenotype_df['treated']].index
time = phenotype_df[phenotype_df['treated']]['hours']
interaction = res[res['fdr_interaction'] < 0.05]
interaction_genes = interaction.index
df = exp_filter.loc[interaction_genes, samples]

popts = fit_trajectories(df, time)

def group_trajectories(s):
    direction = np.sign(s['lin_slope'])
    r2_improvement = s['r2_quad'] - s['r2_linear']
    if r2_improvement > 0.05 and s['quad_importance'] > 0.1:
        if direction * s['q_c'] < 0:
            response = "early_response"
        else:
            response = "late_response"
    else:
        response = "sustained_linear"
    return pd.Series({'direction':  f'{'positive' if direction>0 else 'negative'} response', 'response': response})

interaction_traj_groups = popts.apply(group_trajectories, axis=1)


In [ ]:
unique_response = interaction_traj_groups['response'].unique()
lut_row = dict(zip(unique_response, sns.color_palette("Set1", len(unique_response))))
response_colors = interaction_traj_groups['response'].map(lut_row)

unique_dir = interaction_traj_groups['direction'].unique()
lut_dir = dict(zip(unique_dir, sns.color_palette("Set2", len(unique_dir))))
direction_colors = interaction_traj_groups['direction'].map(lut_dir)
row_colors = pd.DataFrame({'Direction': direction_colors, 'Response': response_colors})

norm = plt.Normalize(time.min(), time.max())
cmap = sns.cubehelix_palette(as_cmap=True)
col_colors_time = time.map(lambda x: cmap(norm(x)))

col_colors = pd.DataFrame({'Time': col_colors_time})

plot_data = df.loc[interaction_traj_groups.sort_values(by=['response', 'direction']).index]
row_colors = row_colors.loc[plot_data.index]

g = sns.clustermap(
    plot_data,
    z_score=0,  # Normalize rows (genes) to Z-scores
    cmap="vlag",  # Blue-White-Red diverging palette
    center=0,  # Center the colormap at Z=0
    col_cluster=False,  # KEEP columns ordered by Time!
    row_cluster=False,  # KEEP rows ordered by Group!
    row_colors=row_colors,  # Add the trajectory bar
    col_colors=col_colors,  # Add Time/Treat bars
    yticklabels=False,  # Hide gene names (too cluttered usually)
    xticklabels=True,
    figsize=(10, 12),
    cbar_pos=(0.02, 0.8, 0.03, 0.15)  # Move colorbar to top left
)

from matplotlib.patches import Patch
handles = [Patch(facecolor=lut_row[name], edgecolor='w', label=name) for name in unique_response]
handles.extend([Patch(facecolor=lut_dir[name], edgecolor='w', label=name) for name in unique_dir])
plt.legend(handles=handles, title='Trajectory', bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.title("Heatmap of Significant Genes (FDR < 0.05)", y=1.05)
plt.show()

Finalize Deliverables

In [ ]:
final_main = main_only[['fdr_treat', 'fdr_interaction']].copy()
final_main['response_class'] = 'treatment'
final_interaction = interaction[['fdr_interaction', 'fdr_treat']].copy()
final_interaction['response_class'] = interaction_traj_groups['response']
final = pd.concat([final_main, final_interaction], axis=0)
supp = popts[['lin_slope', 'peak_time']].copy()
supp['response_class'] = interaction_traj_groups['response']
supp['direction'] = interaction_traj_groups['direction']